In [1]:
#wrapping things up 
#have code that shows we werent able to predict any better than just predicting the dominant class 
#iterating through models by adding and subtracting variables to show colinearity 
#to show that there is no relationship to be found 
#sum things up with little recommendations for future research 
#throw in some charts (crosstabs (box and whisker plot), bar graph (for classes))
#model with different things (but maybe not because throwing kitchen sink is useless)
#run model on six months before and after
#change bottom model to six months before and after 

# Obtaining Data

In [2]:
#Importing libraries needed
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
#opening the datasets 
bottom =  pd.read_csv(r'data\bottom_hurricane.csv')
middle =  pd.read_csv(r'data\middle_hurricane.csv')
top =  pd.read_csv(r'data\top_hurricane.csv')

In [4]:
#let's get rid of duplicate cities by only grabbing the cities with the max value for AWND
bottom = bottom.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
middle = middle.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top = top.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()

In [5]:
bottom.head()

,City,HurricaneName,DATE,AWND,WSF2,SizeRank,b,a,percent,bool
5,Apalachicola,mi,10/10/2018,29.75,63.1,12877,65287.07867,76710.34737,17.496982,1.0
11,Brooksville,ma,10/7/2016,19.69,30.0,1410,50528.88696,60596.57200,19.924613,1.0
20,Clearwater,ma,10/7/2016,20.58,31.1,233,75030.31664,87159.19859,16.165308,1.0
29,Daytona Beach,ma,10/7/2016,27.51,52.1,744,55949.49283,64257.66956,14.849423,0.0
44,Fort Lauderdale,ma,10/7/2016,16.55,29.1,190,138330.70080,154191.06280,11.465540,0.0


In [6]:
middle.head()

,City,HurricaneName,DATE,AWND,WSF2,SizeRank,b,a,percent,bool
5,Apalachicola,mi,10/10/2018,29.75,63.1,12877,144565.87220,161515.92750,11.724797,1.0
11,Brooksville,ma,10/7/2016,19.69,30.0,1410,131341.02820,147305.08070,12.154658,1.0
20,Clearwater,ma,10/7/2016,20.58,31.1,233,164004.34390,180947.34500,10.330825,0.0
29,Cross City,ma,10/7/2016,16.33,27.1,11954,55644.25987,55475.02634,-0.304135,0.0
38,Daytona Beach,ma,10/7/2016,27.51,52.1,744,119402.40490,132585.84080,11.041181,1.0


In [7]:
top.head()

,City,HurricaneName,DATE,AWND,WSF2,SizeRank,b,a,percent,bool
5,Apalachicola,mi,10/10/2018,29.75,63.1,12877,279871.2479,308859.8100,10.357821,1.0
11,Brooksville,ma,10/7/2016,19.69,30.0,1410,236668.7002,254118.0847,7.372916,1.0
20,Clearwater,ma,10/7/2016,20.58,31.1,233,331293.1616,349546.3935,5.509692,0.0
29,Cross City,ma,10/7/2016,16.33,27.1,11954,124107.7381,125538.8548,1.153124,0.0
38,Daytona Beach,ma,10/7/2016,27.51,52.1,744,220281.3651,235594.0105,6.951403,0.0


## Checking Crosstabs

### Bottom Tier Home Values

In [8]:
#check crosstabs 
bottom[bottom['bool'] == 0].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.0
mean,18.691111,37.533333,400.333333,94271.502143,106658.738416,14.294362,0.0
std,4.566811,14.350651,439.485394,32799.627145,34310.857279,6.968502,0.0
min,12.750000,23.000000,16.000000,51493.986580,63037.950880,1.715556,0.0
25%,15.660000,26.700000,107.000000,70686.035723,76881.636387,9.995740,0.0
50%,18.230000,32.100000,204.500000,89851.854570,103229.680500,12.521362,0.0
75%,21.865000,44.300000,626.750000,117165.998800,131188.480800,17.316173,0.0
max,27.510000,76.100000,1644.000000,170218.203700,185409.502800,28.183395,0.0


In [9]:
#check crosstabs 
bottom[bottom['bool'] == 1].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.0
mean,22.369231,38.784615,1862.769231,79915.419922,98993.668878,23.108019,1.0
std,5.117712,13.247191,3532.525823,23642.004600,33024.845069,7.272369,0.0
min,15.660000,21.900000,12.000000,50528.886960,60596.572000,16.165308,1.0
25%,19.460000,30.000000,113.000000,65287.078670,76710.347370,17.624896,1.0
50%,21.250000,32.000000,350.000000,75030.316640,89037.172400,19.924613,1.0
75%,24.830000,53.000000,1410.000000,91758.659330,109114.022000,24.912005,1.0
max,33.330000,63.100000,12877.000000,121735.801300,166366.441400,36.661886,1.0


### Middle Tier Home Values

In [10]:
#check crosstabs 
middle[middle['bool'] == 0].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0
mean,19.637895,37.405263,1106.421053,173096.570156,189599.531107,9.886511,0.0
std,4.937980,13.960280,2716.798449,53256.051020,55216.048302,6.384666,0.0
min,12.750000,23.000000,12.000000,55644.259870,55475.026340,-0.304135,0.0
25%,16.220000,28.100000,108.000000,146272.683200,161420.343200,6.022504,0.0
50%,19.460000,31.100000,219.000000,168090.344800,185504.511900,9.706864,0.0
75%,21.920000,42.500000,810.000000,193946.169250,213005.085900,10.412815,0.0
max,33.330000,76.100000,11954.000000,280584.508000,295369.592000,24.756066,0.0


In [11]:
#check crosstabs 
middle[middle['bool'] == 1].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.0
mean,21.759091,40.836364,1986.727273,165776.665236,198624.407509,18.698109,1.0
std,5.136710,13.071899,3797.605142,45241.977348,65568.232826,9.064933,0.0
min,13.650000,25.100000,20.000000,107890.268300,132585.840800,10.984947,1.0
25%,18.120000,30.000000,206.000000,125371.716550,142445.717550,11.382989,1.0
50%,22.590000,38.000000,529.000000,175538.384000,194821.182600,14.050237,1.0
75%,25.275000,52.550000,1376.500000,195007.514850,228490.642600,25.388068,1.0
max,29.750000,63.100000,12877.000000,237636.125500,316361.491800,33.601156,1.0


### Top Tier Home Values

In [12]:
#check crosstabs 
top[top['bool'] == 0].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.0
mean,20.006087,37.778261,897.695652,313723.715126,337404.854117,7.773596,0.0
std,4.917864,14.052853,2482.388649,104478.805625,109901.459202,6.974185,0.0
min,12.750000,21.900000,12.000000,124107.738100,125538.854800,1.153124,0.0
25%,16.440000,28.100000,97.000000,239055.556650,257389.864800,4.704960,0.0
50%,19.460000,31.100000,190.000000,307648.934500,349546.393500,5.212095,0.0
75%,23.150000,47.550000,547.000000,372860.831400,392491.459900,6.421340,0.0
max,33.330000,76.100000,11954.000000,631061.618900,640267.673900,25.576230,0.0


In [13]:
#check crosstabs 
top[top['bool'] == 1].describe()

,AWND,WSF2,SizeRank,b,a,percent,bool
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.0
mean,21.222500,39.112500,2791.750000,305099.775688,359572.876738,15.623515,1.0
std,5.368697,13.177517,4264.619251,106021.018358,156354.224459,10.540100,0.0
min,13.200000,25.900000,113.000000,173065.912300,206154.635800,7.372916,1.0
25%,18.510000,29.500000,471.500000,228119.932700,245869.213450,7.948337,1.0
50%,21.140000,35.000000,1376.500000,295368.900500,322288.729200,9.784194,1.0
75%,24.212500,45.425000,2262.750000,359250.132475,413269.827050,22.032753,1.0
max,29.750000,63.100000,12877.000000,457389.673300,605350.754900,32.349021,1.0


# Logistic Regression

In [14]:
# For our modeling steps
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss

# For demonstrative pruposes
from scipy.special import logit, expit

In [15]:
logreg = LogisticRegression(random_state=56)

### Iterative Exploration

|hurricane time frame | Increase | No Increase |
|---|---|---|
|one year| 78 | 315 |
|six months| 98 | 295 |
|three months| 84 | 309|
|top| 75 | 326|
|bottom| 183 | 216|

In [16]:
#count for bool = true
print(six['bool'].sum())
#count for bool = false
print(len(six['bool']) - six['bool'].sum())

NameError: name 'six' is not defined

In [ ]:
#checking ratio to see what accuracy is like
295/(295 + 98)

In [ ]:
#count for bool = true
print(one['bool'].sum())
#count for bool = false
print(len(one['bool']) - one['bool'].sum())

In [ ]:
#count for bool = true
print(three['bool'].sum())
#count for bool = false
print(len(three['bool']) - three['bool'].sum())

In [ ]:
#count for bool = true 
print(top['bool'].sum())
#count for bool = false
print(len(top['bool']) - top['bool'].sum())

In [ ]:
#bottom tier homes are more likely to increase in values 
#does before and after a hurricane make a difference?
print(bottom['bool'].sum())
print(len(bottom['bool']) - bottom['bool'].sum())

In [ ]:
#checking ratio to see what accuracy is like
216/(216 + 183)

### Modeling 6 months before and after (whole dataset)

In [ ]:
bottom.head()

In [ ]:
bottom = bottom.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom['bool']
X = bottom.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent', 'WSF2', 'WSF5', 'SizeRank', 'b'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=56)

In [ ]:
logreg.fit(X_train, y_train)
logreg.predict(X_train)

In [ ]:
logreg.classes_

In [ ]:
logreg.predict_proba(X_train)

In [ ]:
#accuracy
logreg.score(X, y, sample_weight=None)

In [ ]:
#accuracy of train
logreg.score(X_train, y_train, sample_weight=None)

In [ ]:
#accuracy of test
logreg.score(X_test, y_test, sample_weight=None)

In [ ]:
logreg.fit(X_test, y_test)
logreg.predict(X_test)

In [ ]:
logreg.get_params(deep=True)

In [ ]:
logreg.coef_

### Modeling 1 Year Before and After

Let's model each hurricane 1 year before and after

#### Modeling hurricane Charley

In [ ]:
#isolating the dataframe to just needed the hurricane
one_c = one[one.HurricaneName == 'c']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
one_c = one_c.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
one_c.head()

In [ ]:
#y is prediction variable 
#X is features 
y = one_c['bool']
X = one_c.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

one_c_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict = {}
model_dict['charley_1year'] = one_c_r
print(model_dict)


#### Modeling hurricane Dennis

In [ ]:
#isolating the dataframe to just needed the hurricane
one_d = one[one.HurricaneName == 'd']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
one_d = one_d.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
one_d.head()

In [ ]:
#y is prediction variable 
#X is features 
y = one_d['bool']
X = one_d.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

one_d_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict['dennis_1year'] = one_d_r

#### Modeling hurricane Matthew

In [ ]:
#isolating the dataframe to just needed the hurricane
one_ma = one[one.HurricaneName == 'ma']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
one_ma = one_ma.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
one_ma.head()

In [ ]:
#y is prediction variable 
#X is features 
y = one_ma['bool']
X = one_ma.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

one_ma_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict['matthew_1year'] = one_ma_r

#### Modeling hurricane Irma

In [ ]:
#isolating the dataframe to just needed the hurricane
one_i = one[one.HurricaneName == 'ir']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
one_i = one_i.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
one_i.head()

In [ ]:
#y is prediction variable 
#X is features 
y = one_i['bool']
X = one_i.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

one_i_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict['irma_1year'] = one_i_r

#### Modeling hurricane Michael

In [ ]:
#isolating the dataframe to just needed the hurricane
one_mi = one[one.HurricaneName == 'mi']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
one_mi = one_mi.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
one_mi.head()

In [ ]:
#y is prediction variable 
#X is features 
y = one_mi['bool']
X = one_mi.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

one_mi_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict['michael_1year'] = one_mi_r

In [ ]:
import json
print(json.dumps(model_dict, indent=4, sort_keys=True, default=str))

### Modeling 6 Months Before and After

Let's model each hurricane 1 year before and after

#### Modeling hurricane Charley

In [ ]:
#isolating the dataframe to just needed the hurricane
six_c = six[six.HurricaneName == 'c']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
six_c = six_c.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
six_c.head()

In [ ]:
#y is prediction variable 
#X is features 
y = six_c['bool']
X = six_c.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

six_c_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict6 = {}
model_dict6['charley_6mo'] = six_c_r

#### Modeling hurricane Dennis

In [ ]:
#isolating the dataframe to just needed the hurricane
six_d = one[one.HurricaneName == 'd']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
six_d = six_d.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
six_d.head()

In [ ]:
#y is prediction variable 
#X is features 
y = six_d['bool']
X = six_d.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

six_d_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict6['dennis_6mo'] = six_d_r

#### Modeling hurricane Matthew

In [ ]:
#isolating the dataframe to just needed the hurricane
six_ma = six[six.HurricaneName == 'ma']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
six_ma = six_ma.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
six_ma.head()

In [ ]:
#y is prediction variable 
#X is features 
y = six_ma['bool']
X = six_ma.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

six_ma_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict6['matthew_6mo'] = six_ma_r

#### Modeling hurricane Irma

In [ ]:
#isolating the dataframe to just needed the hurricane
six_i = six[six.HurricaneName == 'ir']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
six_i = six_i.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
six_i.head()

In [ ]:
#y is prediction variable 
#X is features 
y = six_i['bool']
X = six_i.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

six_i_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict6['irma_6mo'] = six_i_r

#### Modeling hurricane Michael

In [ ]:
#isolating the dataframe to just needed the hurricane
six_mi = six[six.HurricaneName == 'mi']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
six_mi = six_mi.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
six_mi.head()

In [ ]:
#y is prediction variable 
#X is features 
y = six_mi['bool']
X = six_mi.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

six_mi_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict6['michael_6mo'] = six_mi_r

In [ ]:
import json
print(json.dumps(model_dict6, indent=4, sort_keys=True, default=str))

### Modeling 3 Months Before and After

Let's model each hurricane 1 year before and after

#### Modeling hurricane Charley

In [ ]:
#isolating the dataframe to just needed the hurricane
three_c = three[three.HurricaneName == 'c']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
three_c = three_c.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
three_c.head()

In [ ]:
#y is prediction variable 
#X is features 
y = three_c['bool']
X = three_c.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

three_c_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict3 = {}
model_dict3['charley_3mo'] = three_c_r

#### Modeling hurricane Dennis

In [ ]:
#isolating the dataframe to just needed the hurricane
three_d = three[three.HurricaneName == 'd']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
three_d = three_d.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
three_d.head()

In [ ]:
#y is prediction variable 
#X is features 
y = three_d['bool']
X = three_d.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

three_d_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict3['dennis_3mo'] = three_d_r

#### Modeling hurricane Matthew

In [ ]:
#isolating the dataframe to just needed the hurricane
three_ma = three[three.HurricaneName == 'ma']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
three_ma = three_ma.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
three_ma.head()

In [ ]:
#y is prediction variable 
#X is features 
y = three_ma['bool']
X = three_ma.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

three_ma_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict3['matthew_3mo'] = three_ma_r

#### Modeling hurricane Irma

In [ ]:
#isolating the dataframe to just needed the hurricane
three_i = three[three.HurricaneName == 'ir']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
three_i = three_i.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
three_i.head()

In [ ]:
#y is prediction variable 
#X is features 
y = three_i['bool']
X = three_i.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

three_i_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict3['irma_3mo'] = three_i_r

#### Modeling hurricane Michael

In [ ]:
#isolating the dataframe to just needed the hurricane
three_mi = three[three.HurricaneName == 'mi']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
three_mi = three_mi.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
three_mi.head()

In [ ]:
#y is prediction variable 
#X is features 
y = three_mi['bool']
X = three_mi.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

three_mi_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict3['michael_3mo'] = three_mi_r

In [ ]:
import json
print(json.dumps(model_dict3, indent=4, sort_keys=True, default=str))

### Modeling 1 Year Before and After (Top)

Let's model each hurricane 1 year before and after for top tier homes. 

#### Modeling hurricane Charley

In [ ]:
#isolating the dataframe to just needed the hurricane
top_c = top[top.HurricaneName == 'c']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
top_c = top_c.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top_c.head()

In [ ]:
#y is prediction variable 
#X is features 
y = top_c['bool']
X = top_c.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

top_c_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_top = {}
model_dict_top['charley_top'] = top_c_r
print(model_dict_top)


#### Modeling hurricane Dennis

In [ ]:
#isolating the dataframe to just needed the hurricane
top_d = top[top.HurricaneName == 'd']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
top_d = top_d.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top_d.head()

In [ ]:
#y is prediction variable 
#X is features 
y = top_d['bool']
X = top_d.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

top_d_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_top['dennis_top'] = top_d_r

#### Modeling hurricane Matthew

In [ ]:
#isolating the dataframe to just needed the hurricane
top_ma = top[top.HurricaneName == 'ma']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
top_ma = top_ma.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top_ma.head()

In [ ]:
#y is prediction variable 
#X is features 
y = top_ma['bool']
X = top_ma.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

top_ma_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_top['matthew_top'] = top_ma_r

#### Modeling hurricane Irma

In [ ]:
#isolating the dataframe to just needed the hurricane
top_i = top[top.HurricaneName == 'ir']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
top_i = top_i.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top_i.head()

In [ ]:
#y is prediction variable 
#X is features 
y = top_i['bool']
X = top_i.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

top_i_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_top['irma_top'] = top_i_r

#### Modeling hurricane Michael

In [ ]:
#isolating the dataframe to just needed the hurricane
top_mi = top[top.HurricaneName == 'mi']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
top_mi = top_mi.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
top_mi.head()

In [ ]:
#y is prediction variable 
#X is features 
y = top_mi['bool']
X = top_mi.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

top_mi_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_top['michael_top'] = top_mi_r

In [ ]:
import json
print(json.dumps(model_dict_top, indent=4, sort_keys=True, default=str))

### Modeling 1 Year Before and After (Bottom)

Let's model each hurricane 1 year before and after for bottom tier homes. 

#### Modeling hurricane Charley

In [ ]:
#isolating the dataframe to just needed the hurricane
bottom_c = bottom[bottom.HurricaneName == 'c']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
bottom_c = bottom_c.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
bottom_c.head()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom_c['bool']
X = bottom_c.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

bottom_c_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_bottom = {}
model_dict_bottom['charley_bottom'] = bottom_c_r

#### Modeling hurricane Dennis

In [ ]:
#isolating the dataframe to just needed the hurricane
bottom_d = bottom[bottom.HurricaneName == 'd']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
bottom_d = bottom_d.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
bottom_d.head()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom_d['bool']
X = bottom_d.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

bottom_d_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_bottom['dennis_bottom'] = bottom_d_r

#### Modeling hurricane Matthew

In [ ]:
#isolating the dataframe to just needed the hurricane
bottom_ma = bottom[bottom.HurricaneName == 'ma']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
bottom_ma = bottom_ma.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
bottom_ma.head()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom_ma['bool']
X = bottom_ma.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

bottom_ma_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_bottom['matthew_bottom'] = bottom_ma_r

#### Modeling hurricane Irma

In [ ]:
#isolating the dataframe to just needed the hurricane
bottom_i = bottom[bottom.HurricaneName == 'ir']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
bottom_i = bottom_i.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
bottom_i.head()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom_i['bool']
X = bottom_i.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

bottom_i_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_bottom['irma_bottom'] = bottom_i_r

#### Modeling hurricane Michael

In [ ]:
#isolating the dataframe to just needed the hurricane
bottom_mi = bottom[bottom.HurricaneName == 'mi']

#removing duplicate values for city 
#keeping the value that has the highest wind speed
bottom_mi = bottom_mi.sort_values('AWND', ascending=False).drop_duplicates('City').sort_index()
bottom_mi.head()

In [ ]:
#y is prediction variable 
#X is features 
y = bottom_mi['bool']
X = bottom_mi.drop(['bool', 'City', 'HurricaneName', 'DATE', 'a', 'percent'], axis=1)

In [ ]:
logreg.fit(X, y)

bottom_mi_r = logreg.predict(X)

In [ ]:
##create a dictionary to store results of tests
model_dict_bottom['michael_bottom'] = bottom_mi_r

In [ ]:
import json
print(json.dumps(model_dict_bottom, indent=4, sort_keys=True, default=str))

### Looking at all the models 

In [ ]:
print(json.dumps(model_dict, indent=4, sort_keys=True, default=str))
print(json.dumps(model_dict6, indent=4, sort_keys=True, default=str))
print(json.dumps(model_dict3, indent=4, sort_keys=True, default=str))
print(json.dumps(model_dict_top, indent=4, sort_keys=True, default=str))
print(json.dumps(model_dict_bottom, indent=4, sort_keys=True, default=str))